<a href="https://colab.research.google.com/github/samibahig/IFT6390/blob/main/kaggle2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import numpy as np
import scipy.sparse as sp 
import pandas as pd 
# import sklearn.utils.extmath.safe_sparse_dot
import gc
import re, unicodedata
import string
import tqdm
from collections import defaultdict

In [ ]:
train, test = pd.read_csv('/content/train.csv'), pd.read_csv('/content/test.csv')
print (train, test)

FileNotFoundError: ignored

In [ ]:
print(train.keys())

print(train['Abstract'])

train

Index(['Id', 'Abstract', 'Category'], dtype='object')
0         The energy released in a solar flare is part...
1         In light of current atmospheric neutrino osc...
2         We consider the following basic learning tas...
3         In this paper, we characterise the family of...
4         The control of condensed matter systems out ...
                              ...                        
7495      We investigate the dynamical behaviour of a ...
7496      We use the full bispectrum of spherical need...
7497      We show that Ostrogradsky ghosts in higher-d...
7498      McMorris and Powers proved an Arrow-type the...
7499      Recently it has been speculated that the War...
Name: Abstract, Length: 7500, dtype: object


,Id,Abstract,Category
0,0,The energy released in a solar flare is part...,astro-ph
1,1,In light of current atmospheric neutrino osc...,hep-ph
2,2,We consider the following basic learning tas...,cs.LG
3,3,"In this paper, we characterise the family of...",math.CO
4,4,The control of condensed matter systems out ...,cond-mat.mes-hall
...,...,...,...
7495,7495,We investigate the dynamical behaviour of a ...,astro-ph.CO
7496,7496,We use the full bispectrum of spherical need...,astro-ph.CO
7497,7497,We show that Ostrogradsky ghosts in higher-d...,hep-th
7498,7498,McMorris and Powers proved an Arrow-type the...,math.CO


In [ ]:
def remove_punctuation(text):
    pass

def remove_non_ascii(words):
    pass
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        new_words.append(new_word)
        return ''.join(new_words)
def process(df, t):
    df[t] = df[t].apply(lambda x : x.lower())
    #train['Abstract'] = train['Abstract'].apply(lambda x : remove_punctuation(x))
    df[t] = df[t].apply(lambda x : x.strip())
    df[t] = df[t].apply(lambda x : re.sub('\n', ' ', x))
    df[t] = df[t].apply(lambda x : re.sub('\[[^]]*\]', '', x))
    df[t] = df[t].apply(lambda x : re.sub("<.*?>", " ", x))
    df[t] = df[t].apply(lambda x : remove_non_ascii(x))
    df[t] = df[t].str.replace('[^\w\s]','')
    print(df.head())
    return df
    #train['Abstract'] = train['Abstract'].apply(lambda x: remove_punctuation(x))

In [ ]:
t = train['Abstract']

0         The energy released in a solar flare is part...
1         In light of current atmospheric neutrino osc...
2         We consider the following basic learning tas...
3         In this paper, we characterise the family of...
4         The control of condensed matter systems out ...
                              ...                        
7495      We investigate the dynamical behaviour of a ...
7496      We use the full bispectrum of spherical need...
7497      We show that Ostrogradsky ghosts in higher-d...
7498      McMorris and Powers proved an Arrow-type the...
7499      Recently it has been speculated that the War...
Name: Abstract, Length: 7500, dtype: object


In [ ]:
train = process(train, 'Abstract')
test = process(test, 'Abstract')

   Id Abstract           Category
0   0        t           astro-ph
1   1        i             hep-ph
2   2        w              cs.LG
3   3        i            math.CO
4   4        t  cond-mat.mes-hall
   Id Abstract
0   0        w
1   1        t
2   2        o
3   3        g
4   4        t


In [ ]:
class BernoulliVectorizer:
    def __init__(self):
        self.vocab = []
        self.vocab_counter = {}
    def build_vocab(self, data):
        for document in data:
            for word in document.split(' '):
                if word in self.vocab:
                    self.vocab_counter[word] += 1
                else:
                    self.vocab.append(word)
                    self.vocab_counter = 1
    def transform(self, data):
        i = 0 
        for document in data:
            tokens = document.split(' ')  # [ 'my', 'name', 'is', 'kasper', 'and', 'i', 'like', 'ml']
                                          # ['my', 'kasper', 'hello']
            bin_vect = np.zeros(len(self.vocab))
            for word_idx in range(len(self.vocab)):
                for e in tokens: 
                    if e == self.vocab[word_idx]:
                        bin_vect[word_idx ] = 1
                data[i] = bin_vect
                i += 1
        return data
    
    def fit_transform(self, data):
        self.build_vocab(data)
        return transform(data)

B = BernoulliVectorizer()
train['Abstract'] = B.fit_transform(train['Abstract'])
test['Abstract'] = B.transform(test['Abstract'])
train.head()
train, test = train.to_numpy(), test.to_numpy()

class BernoulliNB:
    def __init__(self, alpha):
        self.alpha = alpha
        pass
    '''P(C_k) =  number of documents of that class / number of documents
       P(C_k / w1, w2, ...)\prop P(C_k) P(w1/C_k) P(w2/ C_k)
       P(wi / C_k) = number of documents of class C_k with w_i/ number of documents with that class 
       get all the rows of class C_k, how many of them was word w_i 
       get all the rows of class C_k, how many of them has a 1 in the index of w_i '''
    def fit(self, X, y):
        self.n_classes = len(np.unique(y))
        n_classes = self.n_classes
        self.counts = np.zeros(n_classes)
        self.counts = np.zeros(n_classes)
        for i in y:
            self.counts[i] += 1,
        self.counts /= len(y)
     # generate n_features x n_classes matrix\n",   
        self.params = np.zeros((n_classes, X.shape[1])),
        for idx in range(len(X)):
            self.params[y[idx]] += X[idx]
        self.params += self.alpha 
        class_sums = self.params.sum(axis=1) + self.alpha * self.n_classes
        self.params = self.params / class_sums[:, np.newaxis]
    def predict(self, X): 
        neg_prob = np.log(1 - self.params)
        jll = np.dot(X, (np.log(self.params)-neg_prob).T)           
        jll += np.log(self.counts) + neg_prob.sum(axis=1)
        return np.argmax(jll, axis=1)

NameError: ignored